# Data Collection - Part1

## Load needed Libraries and modules

In [1]:
# import Libs
import pandas as pd
import datetime as dt
import pytz
import os
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# import modules
from datetime import datetime, timezone
from datetime import date, time
from math import trunc
from dateutil.parser import parse

## Collecting Options Data from Yahoo Finance using "yfinance"

### Getting AAPL underlying stock prices

In [2]:
# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
mb_tickerdf04152024 = tickerData.history(period='5y')

# See the data
mb_tickerdf04152024

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-04-15 00:00:00-04:00,47.793186,48.098844,47.656000,47.949623,70146400,0.0,0.0
2019-04-16 00:00:00-04:00,48.004988,48.464674,47.788379,47.954445,102785600,0.0,0.0
2019-04-17 00:00:00-04:00,48.024227,48.948421,47.800402,48.888252,115627200,0.0,0.0
2019-04-18 00:00:00-04:00,48.885853,49.133747,48.741450,49.063953,96783200,0.0,0.0
2019-04-22 00:00:00-04:00,48.816066,49.323890,48.698134,49.225212,77758000,0.0,0.0
...,...,...,...,...,...,...,...
2024-04-09 00:00:00-04:00,168.699997,170.080002,168.350006,169.669998,42451200,0.0,0.0
2024-04-10 00:00:00-04:00,168.800003,169.089996,167.110001,167.779999,49709300,0.0,0.0
2024-04-11 00:00:00-04:00,168.339996,175.460007,168.160004,175.039993,91070300,0.0,0.0


In [3]:
mb_tickerdf04152024.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1259 entries, 2019-04-15 00:00:00-04:00 to 2024-04-15 00:00:00-04:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1259 non-null   float64
 1   High          1259 non-null   float64
 2   Low           1259 non-null   float64
 3   Close         1259 non-null   float64
 4   Volume        1259 non-null   int64  
 5   Dividends     1259 non-null   float64
 6   Stock Splits  1259 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 78.7 KB


### Saving the Stock Prices dataframe into a csv file

In [4]:
# get current date and time
current_datetime = datetime.now()
print("Current date & time : ", current_datetime)
filename1 = datetime.now().strftime("%Y-%m-%d %H-%M-%S-%f")

# create a file object along with extension
file_name = "mb_tickerdf04152024 "+filename1+".csv"
#print (file_name)

# converting to csv
mb_tickerdf04152024.to_csv(file_name)
print("~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~")
print("File saved named:")
print (file_name)
print(current_datetime)    
print("~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~")

Current date & time :  2024-04-15 09:39:43.558192
~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~
File saved named:
mb_tickerdf04152024 2024-04-15 09-39-43-558192.csv
2024-04-15 09:39:43.558192
~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~


### Getting AAPL stock Options related data including Implied Volatilities

In [5]:
# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the option expirations
options_expirations = tickerData.options

# Prepare a list to hold all options data
options_list = []

# Loop through each options expiration date
for expiration in options_expirations:
    # Get options data for each expiration date
    options_data = tickerData.option_chain(expiration)
    
    # Add option type and expiration date to the data
    options_data.calls['OptionType'] = 'Call'
    options_data.calls['expirationDate'] = expiration
    options_data.puts['OptionType'] = 'Put'
    options_data.puts['expirationDate'] = expiration
    
    # Append the data to the options_list
    options_list.append(options_data.calls)
    options_list.append(options_data.puts)

# Concatenate all options data into a single DataFrame
mb_all_options_data04152024 = pd.concat(options_list)

# Reset the index and drop the old one
mb_all_options_data04152024.reset_index(drop=True, inplace=True)

# Convert the expirationDate column to datetime
mb_all_options_data04152024['expirationDate'] = pd.to_datetime(mb_all_options_data04152024['expirationDate'])

# Calculate the time to expiration in days
mb_all_options_data04152024['time_to_expiration'] = (mb_all_options_data04152024['expirationDate'] - datetime.now()).dt.days

# Display the DataFrame
mb_all_options_data04152024

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,OptionType,expirationDate,time_to_expiration
0,AAPL240419C00005000,2024-04-12 13:51:51+00:00,5.0,173.00,0.0,0.0,0.0,0.0,1.0,655,0.00001,True,REGULAR,USD,Call,2024-04-19,3
1,AAPL240419C00010000,2024-03-27 19:35:59+00:00,10.0,163.00,0.0,0.0,0.0,0.0,20.0,11,0.00001,True,REGULAR,USD,Call,2024-04-19,3
2,AAPL240419C00015000,2024-04-04 14:55:59+00:00,15.0,155.95,0.0,0.0,0.0,0.0,1.0,2,0.00001,True,REGULAR,USD,Call,2024-04-19,3
3,AAPL240419C00030000,2024-03-27 15:14:12+00:00,30.0,142.65,0.0,0.0,0.0,0.0,30.0,45,0.00001,True,REGULAR,USD,Call,2024-04-19,3
4,AAPL240419C00035000,2024-04-12 14:02:42+00:00,35.0,142.43,0.0,0.0,0.0,0.0,12.0,3,0.00001,True,REGULAR,USD,Call,2024-04-19,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,AAPL261218P00210000,2024-04-12 17:59:06+00:00,210.0,39.98,0.0,0.0,0.0,0.0,3.0,5,0.00001,True,REGULAR,USD,Put,2026-12-18,976
1911,AAPL261218P00220000,2024-04-11 19:36:50+00:00,220.0,46.72,0.0,0.0,0.0,0.0,2.0,3,0.00001,True,REGULAR,USD,Put,2026-12-18,976
1912,AAPL261218P00230000,2024-04-11 17:17:27+00:00,230.0,58.15,0.0,0.0,0.0,0.0,2.0,3,0.00001,True,REGULAR,USD,Put,2026-12-18,976
1913,AAPL261218P00280000,2024-04-11 18:22:04+00:00,280.0,106.00,0.0,0.0,0.0,0.0,NaN,1,0.00001,True,REGULAR,USD,Put,2026-12-18,976


In [6]:
mb_all_options_data04152024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915 entries, 0 to 1914
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   contractSymbol      1915 non-null   object             
 1   lastTradeDate       1915 non-null   datetime64[ns, UTC]
 2   strike              1915 non-null   float64            
 3   lastPrice           1915 non-null   float64            
 4   bid                 1915 non-null   float64            
 5   ask                 1915 non-null   float64            
 6   change              1915 non-null   float64            
 7   percentChange       1915 non-null   float64            
 8   volume              1848 non-null   float64            
 9   openInterest        1915 non-null   int64              
 10  impliedVolatility   1915 non-null   float64            
 11  inTheMoney          1915 non-null   bool               
 12  contractSize        1915 non-null 

In [7]:
mb_all_options_data04152024.describe().transpose()

,count,mean,min,25%,50%,75%,max,std
strike,1915.0,167.973629,5.0,115.0,165.0,220.0,380.0,74.381553
lastPrice,1915.0,29.221906,0.01,0.4,9.4,47.225,187.0,39.024483
bid,1915.0,7.560392,0.0,0.0,0.0,0.0,195.7,26.319461
ask,1915.0,7.764888,0.0,0.0,0.0,0.0,199.15,26.948647
change,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
percentChange,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
volume,1848.0,795.719697,1.0,2.0,12.0,104.25,150833.0,5857.506331
openInterest,1915.0,3317.779112,0.0,38.0,442.0,2387.0,82242.0,7935.967153
impliedVolatility,1915.0,0.141293,0.00001,0.00001,0.03126,0.125009,10.823245,0.492238
expirationDate,1915,2025-01-23 05:04:32.584856064,2024-04-19 00:00:00,2024-06-21 00:00:00,2024-11-15 00:00:00,2025-06-20 00:00:00,2026-12-18 00:00:00,NaN


### Saving Options Information dataframe into a csv file

In [8]:
# get current date and time
current_datetime = datetime.now()
print("Current date & time : ", current_datetime)
filename1 = datetime.now().strftime("%Y-%m-%d %H-%M-%S-%f")

# create a file object along with extension
file_name = "mb_all_options_data04152024 "+filename1+".csv"
#print (file_name)

# converting to csv
mb_all_options_data04152024.to_csv(file_name)
print("~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~")
print("File saved named:")
print (file_name)
print(current_datetime)    
print("~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~")

Current date & time :  2024-04-15 09:39:46.199775
~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~
File saved named:
mb_all_options_data04152024 2024-04-15 09-39-46-200765.csv
2024-04-15 09:39:46.199775
~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~


### Getting Interest Rate Data

In [9]:
mb_US_T_04152024 = pd.read_csv("US_Treasury_04152024.csv")
mb_US_T_04152024

,US_Treasury_Rates,Yield
0,1m,5.48
1,2m,5.50
2,3m,5.45
3,4m,5.42
4,6m,5.36
5,1y,5.13
6,2y,4.88
7,3y,4.70
8,5y,4.54
9,7y,4.53


### Saving the Interest Rate dataframe into a csv file

In [10]:
# get current date and time
current_datetime = datetime.now()
print("Current date & time : ", current_datetime)
filename1 = datetime.now().strftime("%Y-%m-%d %H-%M-%S-%f")

# create a file object along with extension
file_name = "mb_US_T_04152024 "+filename1+".csv"
#print (file_name)

# converting to csv
mb_US_T_04152024.to_csv(file_name)
print("~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~")
print("File saved named:")
print (file_name)
print(current_datetime)    
print("~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~")

Current date & time :  2024-04-15 09:39:46.240619
~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~
File saved named:
mb_US_T_04152024 2024-04-15 09-39-46-240619.csv
2024-04-15 09:39:46.240619
~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~ ~~~~
